<a href="https://colab.research.google.com/github/krishnamani77/TF2_Notebooks/blob/master/char-rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import keras.utils

sh_url = 'https://homl.info/shakespeare'
sh_file = "sh.txt"
filepath = keras.utils.get_file(sh_file, sh_url)

In [0]:
with open(filepath) as f:
  sh_txt = f.read()  

In [0]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([sh_txt])

In [12]:
print(tokenizer.texts_to_sequences(['Hello world']))
print(tokenizer.sequences_to_texts([[7,2,12,12,4], [1,17,4,9,12,13]]))

[[7, 2, 12, 12, 4, 1, 17, 4, 9, 12, 13]]
['h e l l o', '  w o r l d']


In [13]:
max_id = len(tokenizer.word_index)
print(max_id)

39


In [14]:
print(tokenizer.word_index)

{' ': 1, 'e': 2, 't': 3, 'o': 4, 'a': 5, 'i': 6, 'h': 7, 's': 8, 'r': 9, 'n': 10, '\n': 11, 'l': 12, 'd': 13, 'u': 14, 'm': 15, 'y': 16, 'w': 17, ',': 18, 'c': 19, 'f': 20, 'g': 21, 'b': 22, 'p': 23, ':': 24, 'k': 25, 'v': 26, '.': 27, "'": 28, ';': 29, '?': 30, '!': 31, '-': 32, 'j': 33, 'q': 34, 'x': 35, 'z': 36, '3': 37, '&': 38, '$': 39}


In [0]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([sh_txt]))

In [16]:
print(encoded[10:20])

[36  2 10 24 11 22  2 20  4  9]


In [41]:
print(encoded.__class__)

<class 'numpy.ndarray'>


In [57]:
subseq = encoded[10:20]
subseq = subseq.reshape([subseq.shape[0],1])
print(tokenizer.sequences_to_texts(subseq))

['z', 'e', 'n', ':', '\n', 'b', 'e', 'f', 'o', 'r']


In [17]:
print(len(encoded))

1115394


In [18]:
dataset_size = len(encoded)
train_size = dataset_size * 90 // 100
val_size = dataset_size - train_size
X_train = encoded[:train_size]
X_val = encoded[:val_size]
print(len(X_train), len(X_val))

1003854 111540


In [0]:
n_steps = 100  # number of chars in a sequence
window_length = n_steps + 1  # target is the char just after n_steps.

In [0]:
def get_chunks(sequence, n_steps):
  # 'abcdefghijkl'  when n_steps = 2 should return ['ab','bc','cd','ef',....'jk'] and ['c','d',....'l']
  index = 0
  X = []
  y = []
  while index + n_steps <= len(sequence)-1:
    X.append(sequence[index:index+n_steps])
    y.append(sequence[index+n_steps])
    index += 1
  return X, y

In [31]:
X, y = get_chunks('abcdefghijkl', 3)
print(X, y)

['abc', 'bcd', 'cde', 'def', 'efg', 'fgh', 'ghi', 'hij', 'ijk'] ['d', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l']


In [0]:
X_train, y_train = get_chunks(X_train, 100)
X_val, y_val = get_chunks(X_val, 100)

In [37]:
total_chars = (len(X_train)*100+ len(y_train)+ len(X_val)*100+ len(y_val))
mem = total_chars/1024/1024/1024
print(mem)

0.10489914007484913


In [0]:
#for i in range(10):
#  print(''.join(c for c in tokenizer.sequences_to_texts(X_train[10+i].reshape([X_train[10+i].shape[0],1]))))
#  print(tokenizer.sequences_to_texts(y_train[10+i].reshape([1,1])))